# How to return structured data from a model
- chat models
- function/tool calling
- structured output


参考: [How to return structured data from a model](https://python.langchain.com/docs/how_to/structured_output/)

## chat models

In [1]:
import sys
sys.path.append('/Users/ericyoung/ysx/code/github-study/langChain-rookie')
from env_utils import load_environment_variables

load_environment_variables()

__file__: /Users/ericyoung/ysx/code/github-study/langChain-rookie/env_utils.py
dotenv_path1: /Users/ericyoung/ysx/code/github-study/langChain-rookie/.env
load env ok


### 支持tool的 chat model

In [2]:
# Ollama
from langchain_ollama import ChatOllama
llm = ChatOllama(
    model="qwen2.5:latest", # gemma3:latest(不支持tool)
    temperature=0.7,
    # other params...
)

### 不支持tool的chat model

In [3]:
# 任何chat model
from models import MyOpenAIModel

model = MyOpenAIModel()
response = model.generate("你好，介绍一下你自己")
print(response)

您好！我是 Gemma，一个由 Google DeepMind 训练的大型语言模型。我是一个开放权重的 AI 助手，这意味着我的模型权重是公开的，您可以自由地使用和修改我。

我可以接收文本和图像作为输入，并仅输出文本。 我可以尝试回答你的问题、提供信息、进行创作等等。

由于我是由 Google DeepMind 训练的，因此我的知识截止到 2023 年初。

希望您喜欢与我互动！ 您想聊些什么呢？
您好！我是 Gemma，一个由 Google DeepMind 训练的大型语言模型。我是一个开放权重的 AI 助手，这意味着我的模型权重是公开的，您可以自由地使用和修改我。

我可以接收文本和图像作为输入，并仅输出文本。 我可以尝试回答你的问题、提供信息、进行创作等等。

由于我是由 Google DeepMind 训练的，因此我的知识截止到 2023 年初。

希望您喜欢与我互动！ 您想聊些什么呢？



### ⭐️ 利用 ChatOpenAI 使用自己的base_url (模型本身不支持tool)
- 支持结构化输出, with_structured_output方法
- 支持使用工具, bind_tools方法
- llm = ChatOpenAI(
                model="gpt-4o",
                temperature=0,
                max_tokens=None,
                timeout=None,
                max_retries=2,
                # api_key="...",
                # base_url="...",
                # organization="...",
                # other params...
            )

In [4]:
from models import get_base_url_model_with_tools
llm2 = get_base_url_model_with_tools()
print(llm2.invoke("你是谁?").content)

我是Gemma，一个开放权重的AI助手。我是一个由Google DeepMind训练的大型语言模型。

我是一个由文本和图像组成的模型，主要提供文本输入和输出。 

你可以把我当作一个工具来使用，我可以尝试回答你的问题、生成创意内容等等。


In [7]:
from langchain_core.tools import tool

@tool
def greeting(name: str):
    '''向朋友致欢迎语'''
    return f"你好啊, {name}"

llm_with_tools = llm2.bind_tools([greeting])
llm_with_tools.invoke("你好，我叫tom,希望可以成为你的新朋友")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '875086986', 'function': {'arguments': '{"name":"tom"}', 'name': 'greeting'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 390, 'total_tokens': 420, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gemma-3-4b-it', 'system_fingerprint': 'gemma-3-4b-it', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-877a6d31-646c-4766-a011-1d4121f510ea-0', tool_calls=[{'name': 'greeting', 'args': {'name': 'tom'}, 'id': '875086986', 'type': 'tool_call'}], usage_metadata={'input_tokens': 390, 'output_tokens': 30, 'total_tokens': 420, 'input_token_details': {}, 'output_token_details': {}})

## .with_structured_output() 结构化输出
- 前置条件: 需要**支持结构化方法的模型**才可以,见 https://python.langchain.com/docs/integrations/chat/ ;
- 例如, MyOpenAIModel 就无法支持, ChatOllama是支持的

### 使用Pydantic 进行结构化输出

In [5]:
# Pydantic class
from typing import Optional
from pydantic import BaseModel, Field


# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )


structured_llm2 = llm2.with_structured_output(Joke)

structured_llm2.invoke("Tell me a joke about women and answer in chinese")

Joke(setup='Why did the woman break up with the mathematician?', punchline='Because he kept trying to find her area!', rating=3)

- 练习

In [5]:
from typing import Optional
from pydantic import BaseModel, Field

class JenkinsJob(BaseModel):
    """build a Jenkins job info"""
    job_name: str = Field(description="The name of the job")
    run_result: str = Field(description="The result of the job")
    build_number: Optional[str] = Field(description="The build number of the job, such as #1104")

structured_llm2 = llm.with_structured_output(JenkinsJob)

structured_llm2.invoke("build a jenkins ci job and answer in chinese")

JenkinsJob(job_name='example-job', run_result='SUCCESS', build_number='#1')

### 使用 TypedDict 进行结构化输出
- Requirements
    - Core: langchain-core>=0.2.26
    - Typing 扩展：强烈建议从 typing_extensions 导入 Annotated 和 TypedDict ，而不是 typing ，以确保在不同版本的 Python 中行为一致。

In [6]:
from typing import Optional
from typing_extensions import Annotated, TypedDict


# TypedDict
class Joke(TypedDict):
    """Joke to tell user."""

    setup: Annotated[str, ..., "The setup of the joke"]

    # Alternatively, we could have specified setup as:

    # setup: str                    # no default, no description
    # setup: Annotated[str, ...]    # no default, no description
    # setup: Annotated[str, "foo"]  # default, no description

    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]


structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")

{'punchline': 'Because it loved to be a purr-fessional!',
 'rating': 8,
 'setup': 'Why did the cat join the rock band?'}

### 使用json schema进行结构化输出

In [7]:
json_schema = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            "description": "How funny the joke is, from 1 to 10",
            "default": None,
        },
    },
    "required": ["setup", "punchline"],
}
structured_llm = llm.with_structured_output(json_schema)

structured_llm.invoke("Tell me a joke about cats")

{'punchline': 'Because there are too many cheetahs!',
 'rating': 8,
 'setup': "Why don't cats play poker in the jungle?"}

### multiple schemas 进行结构化输出
- 联合多个schemas,LLM通过用户的输入选择合适的schema进结构化输出
- 支持Pydantic/TypedDict/json schema三种方式构造

In [10]:
from typing import Union


class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )


class ConversationalResponse(BaseModel):
    """Respond in a conversational manner. Be kind and helpful."""

    response: str = Field(description="A conversational response to the user's query")


class FinalResponse(BaseModel):
    final_output: Union[Joke, ConversationalResponse]


structured_llm2 = llm.with_structured_output(FinalResponse)

structured_llm2.invoke("Tell me a joke about cats")

In [12]:
structured_llm2.invoke("How are you today? must answer to me")

## steaming 流式输出
- 结构化输出模型的流式输出,尤其是TypedDict class or JSON Schema dict

In [17]:
from typing_extensions import Annotated, TypedDict


# TypedDict
class Joke(TypedDict):
    """Joke to tell user."""

    setup: Annotated[str, ..., "The setup of the joke"]
    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]


structured_llm = llm.with_structured_output(Joke)

# 没有流式效果?????
for chunk in structured_llm.stream("Tell me a joke about cats"):
    print(chunk)

{'punchline': 'Because they make up everything!', 'rating': 8, 'setup': "Why don't scientists trust atoms?"}


## Few-shot prompting
- 对于更复杂的模式，向提示中添加少量示例非常有用

In [25]:
from langchain_core.prompts import ChatPromptTemplate

system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") and the final punchline (the response to "<setup> who?"), and rating.

Here are some examples of jokes:

example_user: Tell me a joke about planes
example_assistant: {{"setup": "Why don't planes ever get tired?", "punchline": "Because they have rest wings!", "rating": 2}}

example_user: Tell me another joke about planes
example_assistant: {{"setup": "Cargo", "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!", "rating": 10}}

example_user: Now about caterpillars
example_assistant: {{"setup": "Caterpillar", "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!", "rating": 5}}"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")])

few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke("what's something funny about woodpeckers")

Joke(setup='Woodpecker', punchline="Woodpecker drills holes in trees, but his wife always says 'hole-y woodpecker, poor bird!'", rating=8)

### few-shot的结构化输出通过工具调用
- 当底层方法用于**结构化输出是通过工具调用时**，我们可以将示例作为显式的工具调用传递进去。
- 你可以检查你使用的模型在其 API 参考中是否使用了工具调用。
- 详见: https://python.langchain.com/docs/how_to/tools_few_shot/
- 运行报错????

In [29]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

examples = [
    HumanMessage("Tell me a joke about planes", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Why don't planes ever get tired?",
                    "punchline": "Because they have rest wings!",
                    "rating": 2,
                },
                "id": "1",
            }
        ],
    ),
    # Most tool-calling models expect a ToolMessage(s) to follow an AIMessage with tool calls.
    ToolMessage("", tool_call_id="1"),
    # Some models also expect an AIMessage to follow any ToolMessages,
    # so you may need to add an AIMessage here.
    HumanMessage("Tell me another joke about planes", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Cargo",
                    "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!",
                    "rating": 10,
                },
                "id": "2",
            }
        ],
    ),
    ToolMessage("", tool_call_id="2"),
    HumanMessage("Now about caterpillars", name="example_user"),
    AIMessage(
        "",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Caterpillar",
                    "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!",
                    "rating": 5,
                },
                "id": "3",
            }
        ],
    ),
    ToolMessage("", tool_call_id="3"),
]
system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") \
and the final punchline (the response to "<setup> who?")."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("placeholder", "{examples}"), ("human", "{input}")]
)
few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke({"input": "crocodiles", "examples": examples})

## (高级用法) 指定输出结构化的方法
- 对于支持多种输出结构化方式的模型（即，它们**同时支持工具调用和 JSON 模式**），可以通过 method= 参数指定使用哪种方法

In [32]:
# 设置json模式
structured_llm = llm.with_structured_output(None, method="json_mode")

# 指定输出哪些key
structured_llm.invoke(
    "Tell me a joke about cats, respond in JSON with `setup` and `punchline` , `rating` keys"
)

{'setup': "Why don't cats play poker in the jungle?",
 'punchline': 'Because there are too many cheetahs!',
 'rating': 'G'}

## (高级用法) Raw outputs  原始输出
- LLMs 并不擅长生成结构化输出的，尤其是当模式变得复杂时。
- 你可以通过传递 include_raw=True 来避免抛出异常并自行处理原始输出。
- 输出的response中包含 原始消息输出(raw)、 解析成功的结果(parsed) , 解析错误的信息(parsing_error)：

In [35]:
structured_llm = llm.with_structured_output(Joke, include_raw=True)

res = structured_llm.invoke("Tell me a joke about cats")
print(res)
print("\nraw====\n")
print(res['raw'])
print("\nparsed====\n")
print(res['parsed'])
print("\nparsing_error====\n")
print(res['parsing_error'])

{'raw': AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:latest', 'created_at': '2025-03-15T14:47:49.315938Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4955127084, 'load_duration': 44055584, 'prompt_eval_count': 204, 'prompt_eval_duration': 2522000000, 'eval_count': 50, 'eval_duration': 2383000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-cadc5235-c910-4cfc-b4dc-21a435901627-0', tool_calls=[{'name': 'Joke', 'args': {'punchline': 'Because there are too many cheetahs!', 'rating': 8, 'setup': "Why don't cats play poker in the jungle?"}, 'id': '278bad2b-6217-474b-ac43-ebceec97d7fb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 204, 'output_tokens': 50, 'total_tokens': 254}), 'parsed': Joke(setup="Why don't cats play poker in the jungle?", punchline='Because there are too many cheetahs!', rating=8), 'parsing_error': None}

raw====

content='' additional_kwargs={} response_metadata={'model

## ⭐️(高级用法) 不支持tool调用的模型解析模型输出
- 直接提示并解析模型输出
- 并非所有模型都支持 .with_structured_output() ，因为并非所有模型都具有工具调用或 JSON 模式支持。
- 对于此类模型，您需要**直接提示模型使用特定格式，并使用输出解析器从原始模型输出中提取结构化响应**。

### 使用 PydanticOutputParser
- 以下示例使用内置的 PydanticOutputParser 来解析由提示匹配给定 Pydantic 模式的聊天模型输出。
- 请注意，我们直接从解析器的方法中向提示添加了 format_instructions
- 输出解析器与提示技术生成结构化输出的深入介绍, 参考: https://python.langchain.com/docs/how_to/output_parser_structured/

In [37]:
from typing import List

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person")
    height_in_meters: float = Field(
        ..., description="The height of the person expressed in meters."
    )


class People(BaseModel):
    """Identifying information about all people in a text."""

    people: List[Person]


# Set up a parser
parser = PydanticOutputParser(pydantic_object=People)

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

query = "Anna is 23 years old and she is 6 feet tall"

print(prompt.invoke({"query": query}).to_string())

System: Answer the user query. Wrap the output in `json` tags
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Person": {"description": "Information about a person.", "properties": {"name": {"description": "The name of the person", "title": "Name", "type": "string"}, "height_in_meters": {"description": "The height of the person expressed in meters.", "title": "Height In Meters", "type": "number"}}, "required": ["name", "height_in_meters"], "title": "Person", "type": "object"}}, "description": "Identifying information about all people in a text.", "properties": {"people": {"items"

In [46]:
chain = prompt | llm | parser

chain.invoke({"query": query})

People(people=[Person(name='Anna', height_in_meters=1.8288)])

### Custom Parsing  自定义解析
- 您还可以使用 LangChain Expression Language ([LCEL](https://python.langchain.com/docs/concepts/lcel/)) 创建自定义提示和解析器，并使用普通函数解析模型的输出

In [44]:
import json
import re
from typing import List

from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person")
    height_in_meters: float = Field(
        ..., description="The height of the person expressed in meters."
    )


class People(BaseModel):
    """Identifying information about all people in a text."""

    people: List[Person]


# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Output your answer as JSON that  "
            r"matches the given schema: \`\`\`json\n{schema}\n\`\`\`. "
            r"Make sure to wrap the answer in \`\`\`json and \`\`\` tags",
        ),
        ("human", "{query}"),
    ]
).partial(schema=People.model_json_schema())


# Custom parser
def extract_json(message: AIMessage) -> List[dict]:
    r"""Extracts JSON content from a string where JSON is embedded between \`\`\`json and \`\`\` tags.

    Parameters:
        text (str): The text containing the JSON content.

    Returns:
        list: A list of extracted JSON strings.
    """
    text = message.content
    # Define the regular expression pattern to match JSON blocks
    pattern = r"\`\`\`json(.*?)\`\`\`"

    # Find all non-overlapping matches of the pattern in the string
    matches = re.findall(pattern, text, re.DOTALL)

    # Return the list of matched JSON strings, stripping any leading or trailing whitespace
    try:
        return [json.loads(match.strip()) for match in matches]
    except Exception:
        raise ValueError(f"Failed to parse: {message}")

In [52]:
chain = prompt | llm

res = chain.invoke({"query": query})
print("\nres====\n")
print(res)
print("\nparsed====\n")
parsed = extract_json(res)
print(parsed)


res====

content='```json\n{\n  "people": [\n    {\n      "name": "Anna",\n      "height_in_meters": 1.8288\n    }\n  ]\n}\n```' additional_kwargs={} response_metadata={'model': 'qwen2.5:latest', 'created_at': '2025-03-15T15:12:18.762255Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3836013458, 'load_duration': 42675333, 'prompt_eval_count': 242, 'prompt_eval_duration': 1906000000, 'eval_count': 40, 'eval_duration': 1880000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-1f1688ab-b26b-44c9-a8c1-410c29e9fb01-0' usage_metadata={'input_tokens': 242, 'output_tokens': 40, 'total_tokens': 282}

parsed====

[{'people': [{'name': 'Anna', 'height_in_meters': 1.8288}]}]
